In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import

df=pd.read_csv("/content/Iris.csv")
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

df.head()
df.info()
df.shape
df.describe()
df.isnull().sum()

# convert the species into each respective category to be feed into the neural network

print(df["Species"].unique())

df.loc[df["Species"]=="Iris-setosa","Species"]=0
df.loc[df["Species"]=="Iris-versicolor","Species"]=1
df.loc[df["Species"]=="Iris-virginica","Species"]=2
print(df.head())

df=df.iloc[np.random.permutation(len(df))]
print(df.head())

# Convert class labels to integers
class_mapping = {label: idx for idx, label in enumerate(np.unique(df["Species"]))}
#class_mapping = {"setosa": 0, "versicolor": 1, "virginica": 2}

df["Species"] = df["Species"].map(class_mapping)


# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)


# Split the dataset into training and testing sets
train_size = int(0.7 * len(df))
train_features = df.iloc[:train_size, :-1].values
train_labels = df.iloc[:train_size, -1].values
test_features = df.iloc[train_size:, :-1].values
test_labels = df.iloc[train_size:, -1].values


# Define the neural network architecture
class NeuralNetwork:
    def __init__(self, n_input, n_hidden, n_output):
        self.weights_hidden = np.random.randn(n_input, n_hidden)
        self.bias_hidden = np.zeros((1, n_hidden))
        self.weights_output = np.random.randn(n_hidden, n_output)
        self.bias_output = np.zeros((1, n_output))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)




    def feedforward(self, x):
        self.hidden_layer = np.dot(x, self.weights_hidden) + self.bias_hidden
        self.hidden_layer= self.relu(self.hidden_layer)
        self.output_layer = np.dot(self.hidden_layer, self.weights_output) + self.bias_output
        self.output_layer= self.sigmoid( self.output_layer)
        return self.output_layer
    
    def backpropagation(self, x, y, output):
        error = y - output
        output_delta = error * self.sigmoid_derivative(output)
        hidden_error = np.dot(output_delta, self.weights_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_layer)
        
        self.weights_hidden += np.dot(x.T, hidden_delta)
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True)
        self.weights_output += np.dot(self.hidden_layer.T, output_delta)
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True)
# Train the neural network
n_input = train_features.shape[1]
n_hidden = 5
n_output =  len(np.unique(train_labels))
nn = NeuralNetwork(n_input, n_hidden, n_output)

epochs = 500
learning_rate = 0.0001

for i in range(epochs):
    output = nn.feedforward(train_features)
    nn.backpropagation(train_features, np.eye(n_output)[train_labels], output)
   
   